# Loop 24 LB Feedback Analysis

**Latest submission**: exp_025 scored 70.6261 on LB (CV = LB exactly, as expected for deterministic problem)

**Key insight**: After 25 experiments, total improvement is only 0.021 points. Gap to target is 1.707 points (2.42%).

## Critical Analysis

The evaluator correctly identifies that:
1. All incremental optimization approaches converge to the same local optimum
2. The improvement rate (~0.0008 per experiment) is far too slow
3. We need a FUNDAMENTALLY DIFFERENT approach, not more incremental optimization

In [ ]:
import pandas as pd
import numpy as np
import json

# Load session state to understand experiment history
with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

# Analyze experiment progression
experiments = state['experiments']
print(f"Total experiments: {len(experiments)}")
print(f"\nScore progression:")
for exp in experiments:
    print(f"  {exp['name']}: {exp['score']:.6f}")

In [ ]:
# Calculate improvement rate
scores = [exp['score'] for exp in experiments]
best_score = min(scores)
initial_score = scores[0]

print(f"Initial score: {initial_score:.6f}")
print(f"Best score: {best_score:.6f}")
print(f"Total improvement: {initial_score - best_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap to target: {best_score - 68.919154:.6f}")
print(f"Improvement rate: {(initial_score - best_score) / len(experiments):.6f} per experiment")
print(f"\nExperiments needed at current rate: {(best_score - 68.919154) / ((initial_score - best_score) / len(experiments)):.0f}")

In [ ]:
# Analyze per-N scores to find where improvements might be possible
df = pd.read_csv('/home/submission/submission.csv')

def parse_s_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

df['x'] = df['x'].apply(parse_s_value)
df['y'] = df['y'].apply(parse_s_value)
df['deg'] = df['deg'].apply(parse_s_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Submission loaded: {len(df)} rows")

In [ ]:
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate

TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(trees):
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def calculate_score(positions, n):
    trees = [create_tree_polygon(p[0], p[1], p[2]) for p in positions]
    side = get_bounding_box_side(trees)
    return (side ** 2) / n

# Calculate per-N scores
per_n_scores = {}
for n in range(1, 201):
    group = df[df['n'] == n]
    positions = np.array([[row['x'], row['y'], row['deg']] for _, row in group.iterrows()])
    per_n_scores[n] = calculate_score(positions, n)

print(f"Total score: {sum(per_n_scores.values()):.6f}")

In [ ]:
# Analyze efficiency by N (lower is better)
# Theoretical minimum for N trees is approximately 0.35 (based on tree area)
tree_area = Polygon(TREE_TEMPLATE).area
print(f"Single tree area: {tree_area:.6f}")

# Calculate efficiency = actual_score / theoretical_minimum
# Theoretical minimum = N * tree_area / N = tree_area (if perfect packing)
efficiencies = {}
for n, score in per_n_scores.items():
    # Score = side^2 / n, so side = sqrt(score * n)
    # Efficiency = actual_area / (n * tree_area) = score / tree_area
    efficiencies[n] = score / tree_area

# Find N values with worst efficiency (most room for improvement)
worst_efficiency = sorted(efficiencies.items(), key=lambda x: -x[1])[:20]
print("\nN values with WORST efficiency (most room for improvement):")
for n, eff in worst_efficiency:
    print(f"  N={n}: efficiency={eff:.4f}, score={per_n_scores[n]:.6f}")

In [ ]:
# Analyze score contribution by N range
ranges = [
    (1, 10, "N=1-10"),
    (11, 20, "N=11-20"),
    (21, 50, "N=21-50"),
    (51, 100, "N=51-100"),
    (101, 150, "N=101-150"),
    (151, 200, "N=151-200")
]

print("\nScore contribution by N range:")
for start, end, label in ranges:
    range_score = sum(per_n_scores[n] for n in range(start, end+1))
    pct = range_score / sum(per_n_scores.values()) * 100
    print(f"  {label}: {range_score:.4f} ({pct:.1f}%)")
    
# Large N (100-200) contributes most to total score
# But small N has worst efficiency - which is the better target?

In [ ]:
# Key insight from research:
# 1. MIP can PROVE optimality for small N
# 2. Top teams use "exact geometric optimization" and "high-precision arithmetic"
# 3. The baseline may not be globally optimal

# Let's check if there's any pattern in the N values that improved
# From exp_025: N=87 improved by 0.001494

print("Analysis of N=87:")
print(f"  Score: {per_n_scores[87]:.6f}")
print(f"  Efficiency: {efficiencies[87]:.4f}")
print(f"  Rank by efficiency: {sorted(efficiencies.keys(), key=lambda x: -efficiencies[x]).index(87) + 1}/200")

# What's special about N=87?
# It's not a perfect square, not a triangular number
# Let's check if it's related to tessellation patterns

## Key Findings

1. **Improvement rate is too slow**: At ~0.0008 per experiment, we'd need 2000+ experiments to reach target

2. **Small N has worst efficiency but may already be optimal**: N=1-10 have efficiency 0.53-0.93, but exhaustive search confirmed they're optimal

3. **Large N contributes most to total score**: N=100-200 contributes ~50% of total score

4. **The deletion cascade found ONE improvement (N=87)**: This suggests the baseline is NOT globally optimal for all N

## Strategic Options

### Option 1: MIP for Small N (Evaluator's recommendation)
- Use OR-Tools to formulate exact MIP for N=2-10
- Can PROVE optimality or find improvements
- Risk: Small N may already be optimal (exhaustive search suggests this)

### Option 2: Focus on Large N with Different Initial Structures
- Large N contributes most to score
- Current baseline uses specific structure
- Try fundamentally different arrangements (hexagonal, spiral, etc.)

### Option 3: SparroWASM External Solver
- Professional 2D nesting solver
- Can generate fundamentally different solutions
- May find configurations the baseline doesn't have

### Option 4: Systematic Deletion Cascade
- N=87 improved - run more thorough cascade
- Test ALL N values, try removing EACH tree from N+1
- O(N²) but may find more improvements

In [ ]:
# Let's analyze what makes N=87 special
# Check if there are other N values near 87 that might also benefit

print("Scores around N=87:")
for n in range(80, 95):
    print(f"  N={n}: score={per_n_scores[n]:.6f}, efficiency={efficiencies[n]:.4f}")

# Check if N=87 is related to any mathematical pattern
import math
print(f"\nN=87 analysis:")
print(f"  sqrt(87) = {math.sqrt(87):.4f}")
print(f"  87/6 = {87/6:.4f} (hexagonal rows)")
print(f"  87/7 = {87/7:.4f} (7 rows)")
print(f"  87/8 = {87/8:.4f} (8 rows)")
print(f"  87/9 = {87/9:.4f} (9 rows)")